In [20]:
import pandas as pd
import requests
import json
import time
import os
from functools import reduce

# --- CONFIGURATION ---
LOCAL_CSV_PATH = 'all_countries.csv'
WB_JSON_PATH = 'school_enrollment.json'
UNDP_JSON_PATH = 'region_index.json'
OUTPUT_CSV_PATH = 'final_merged_data.csv' # New constant for output file
WB_BASE_URL = 'https://api.worldbank.org/v2/country/all/indicator/SE.ENR.PRSC.FM.ZS?format=json&date=2024'
UNDP_URL = 'https://api.open.undp.org/api/region-index.json'
WB_PAGES_LIMIT = 6 # Max pages to fetch

def extract_data(
    local_csv_path: str = LOCAL_CSV_PATH,
    wb_base_url: str = WB_BASE_URL,
    wb_pages_limit: int = WB_PAGES_LIMIT,
    wb_json_write_path: str = WB_JSON_PATH,
    undp_url: str = UNDP_URL,
    undp_json_write_path: str = UNDP_JSON_PATH
) -> dict:
    """
    Handles fetching data from external APIs and saves API results to local JSON files.
    
    The local CSV path is included in the return dictionary as it is an input
    required for the subsequent transformation step.

    Returns:
        dict: Paths to the extracted (JSON) and input (CSV) files.
    """
    print("--- Starting Data Extraction Process ---")
    
    # NOTE: The local CSV is not read or checked in this function;
    # it is assumed to be an existing input file for the transformation step.

    # 1. Extract: World Bank API Data with Pagination
    print(f"\n--- Fetching World Bank Data (up to {wb_pages_limit} pages) ---")
    all_wb_data = []
    page = 1
    while page <= wb_pages_limit:
        params = {'page': page, 'per_page': 50}
        try:
            response = requests.get(wb_base_url, params=params, timeout=10)
            if response.status_code != 200:
                print(f"Failed to fetch page {page}: Status {response.status_code}. Breaking loop.")
                break

            data = response.json()
            # World Bank API returns a list [metadata, data_list]
            if len(data) < 2 or not data[1]:
                print(f"No more data on page {page}.")
                break

            all_wb_data.extend(data[1])
            print(f"Fetched page {page} with {len(data[1])} records.")

            total_pages = data[0].get('pages', wb_pages_limit)
            if page >= total_pages:
                break

            page += 1
            time.sleep(0.5) # Be polite to the API

        except requests.exceptions.RequestException as e:
            print(f"Request error on page {page}: {e}. Stopping World Bank extraction.")
            break
        except Exception as e:
            print(f"An unexpected error occurred during World Bank fetch: {e}")
            break

    # Save World Bank data
    if all_wb_data:
        with open(wb_json_write_path, 'w', encoding='utf-8') as f:
            json.dump(all_wb_data, f, indent=4)
        print(f"Saved {len(all_wb_data)} total records to {wb_json_write_path}")
    else:
        print("No World Bank data fetched to save.")


    # 2. Extract: UNDP API Data (Single Request)
    print("\n--- Fetching UNDP Region Index Data ---")
    try:
        response = requests.get(undp_url, timeout=10)
        if response.status_code == 200:
            undp_data = response.json()
            print("Successfully fetched UNDP data.")

            # Save UNDP data
            with open(undp_json_write_path, 'w', encoding='utf-8') as f:
                json.dump(undp_data, f, indent=4)
            print(f"Saved records to {undp_json_write_path}")
        else:
            print(f"Failed to fetch UNDP data. Status code: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Request error for UNDP data: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during UNDP fetch: {e}")

    print("\n--- Data Extraction Complete ---")
    return {
        'countries_csv': local_csv_path,
        'school_enrollment_json': wb_json_write_path,
        'region_index_json': undp_json_write_path
    }

def transform_and_merge_data(file_paths: dict) -> pd.DataFrame:
    """
    Loads local files, transforms them, and performs a full outer merge.

    Args:
        file_paths: Dictionary containing paths to the three required data files.

    Returns:
        A single, fully merged pandas DataFrame.
    """
    print("\n--- Starting Data Transformation and Merge Process ---")

    # --- 1. Load all_countries.csv ---
    try:
        all_countries_df = pd.read_csv(file_paths['countries_csv'])
        print(f"Loaded all_countries_df: {all_countries_df.shape[0]} rows.")
    except Exception as e:
        # This is where the error handling for the missing CSV now resides.
        print(f"ERROR: Could not load countries CSV at {file_paths['countries_csv']}: {e}")
        return pd.DataFrame()

    # --- 2. Transform region_index.json (Complex Flattening) ---
    try:
        with open(file_paths['region_index_json'], 'r', encoding='utf-8') as f:
            region_data = json.load(f)

        # 2a. Normalize base and 'aggregate' keys
        df_base = pd.json_normalize(
            region_data,
            sep='_',
            record_path=None,
            meta=['id', 'name', 'countries']
        )

        # 2b. Explode the 'countries' list
        df_exploded = df_base.explode('countries').reset_index(drop=True)

        # 2c. Flattens the resulting 'countries' column
        df_countries = pd.json_normalize(df_exploded['countries']).add_prefix('country_')

        # 2d. Combine DataFrames
        region_index_df = pd.concat([
            df_exploded.drop('countries', axis=1),
            df_countries
        ], axis=1)
        print(f"Transformed region_index_df: {region_index_df.shape[0]} rows.")

    except Exception as e:
        print(f"ERROR: Failed to transform region index JSON: {e}")
        return pd.DataFrame()

    # --- 3. Transform school_enrollment.json (Simple Flattening) ---
    try:
        with open(file_paths['school_enrollment_json'], 'r', encoding='utf-8') as f:
            school_enrollment_json = json.load(f)

        # Normalize the simple nested structure
        school_enrollment_df = pd.json_normalize(school_enrollment_json, sep='_')
        print(f"Transformed school_enrollment_df: {school_enrollment_df.shape[0]} rows.")

    except Exception as e:
        print(f"ERROR: Failed to transform school enrollment JSON: {e}")
        return pd.DataFrame()


    # --- 4. MERGE DATASETS (Full Outer Joins) ---
    print("\nPerforming sequential full outer merges...")

    # Define the DataFrames and the required join keys
    merge_sequence = [
        (school_enrollment_df, 'countryiso3code'),
        (region_index_df, 'country_code')
    ]

    # Use reduce for sequential merging starting with all_countries_df
    # Note: 'iso3' is the common column in all_countries_df
    df_final = reduce(
        lambda left, right_tuple: pd.merge(
            left,
            right_tuple[0],
            left_on='iso3',
            right_on=right_tuple[1],
            how='outer'
        ),
        merge_sequence,
        all_countries_df
    )

    print("Data transformation and merging complete.")
    return df_final

def load_data(df: pd.DataFrame, output_path: str) -> None:
    """
    Saves the processed DataFrame to a CSV file.

    Args:
        df: The pandas DataFrame to save.
        output_path: The file path for the output CSV.
    """
    if df.empty:
        print("Skipping save: DataFrame is empty.")
        return
        
    try:
        # index=False prevents pandas from writing the DataFrame index as a column
        df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"\n--- Data Load Complete ---")
        print(f"Successfully saved {df.shape[0]} rows to {output_path}")
    except Exception as e:
        print(f"ERROR: Failed to save DataFrame to CSV: {e}")


if __name__ == '__main__':
    # --- Execute the Data Pipeline ---
    
    # Note: To run this successfully, a file named 'all_countries.csv' 
    # must exist in the same directory, containing an 'iso3' column.
    
    # 1. Extraction: Fetch API data and save files
    file_paths = extract_data()
    
    # 2. Transformation and Merging: Load files and combine them
    final_df = transform_and_merge_data(file_paths)
    
    if not final_df.empty:
        print("\n--- Final Merged DataFrame Summary ---")
        print(f"Shape: {final_df.shape}")
        print("\nFirst 5 Rows:")
        # Use to_string() to ensure full output is visible in the console
        print(final_df.head().to_string())
        
        # 3. Loading: Save the final DataFrame to the configured path
        load_data(final_df, OUTPUT_CSV_PATH)
        
        # Drop redundant merge key columns
        final_df = final_df.drop(columns=['countryiso3code', 'country_code'])
        print("\nFinal Columns after dropping redundant keys:")
        print(final_df.columns.tolist())

--- Starting Data Extraction Process ---

--- Fetching World Bank Data (up to 6 pages) ---
Fetched page 1 with 50 records.
Fetched page 2 with 50 records.
Fetched page 3 with 50 records.
Fetched page 4 with 50 records.
Fetched page 5 with 50 records.
Fetched page 6 with 16 records.
Saved 266 total records to school_enrollment.json

--- Fetching UNDP Region Index Data ---
Successfully fetched UNDP data.
Saved records to region_index.json

--- Data Extraction Complete ---

--- Starting Data Transformation and Merge Process ---
Loaded all_countries_df: 250 rows.
Transformed region_index_df: 240 rows.
Transformed school_enrollment_df: 266 rows.

Performing sequential full outer merges...
Data transformation and merging complete.

--- Final Merged DataFrame Summary ---
Shape: (372, 33)

First 5 Rows:
         country     capital    region     continents       area  population iso2 iso3 countryiso3code  date value unit obs_status  decimal       indicator_id                                   